In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import functools
import tempfile

import numpy as np
import pytorch_lightning as pl
import torch
import datamol as dm

import goli

Using backend: pytorch
/home/hadim/local/conda/envs/goli/lib/python3.8/site-packages/umap/__init__.py:9: UserWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn("Tensorflow not installed; ParametricUMAP will be unavailable")


In [2]:
# Load a default config
config = goli.load_config(name="micro_zinc_default")

# Setup a temporary cache file. Only for
# demo purposes, use a known path in prod.
cache_data_path = pathlib.Path(tempfile.mkdtemp()) / "cache.pkl"
cache_data_path = None

# Load a dataframe
df = goli.data.load_tiny_zinc()
df.head()

,SMILES,SA,logp,score
0,CCc1ccc(C(=O)/C(=C(/S)NC2CC2)[n+]2ccc(CC)cc2)cc1,2.775189,3.7897,6.564889
1,C=CCOc1ccc(C(F)(F)F)cc1C(=O)NC[C@H]1CCC[C@H]1O,3.071498,3.1610,6.232498
2,COc1cc(OC)cc([C@@H](NC(=O)Cn2ccccc2=O)c2nccn2C)c1,2.840009,1.5048,4.344809
3,CN1CCC[C@@]2(CCN(C(=O)CN3CCNC(=O)C3)C2)C1=O,3.605168,-1.1109,2.494268
4,COc1ccc(Cl)cc1NC(=O)c1nn(C)cc1[N+](=O)[O-],2.132665,2.2426,4.375265


In [3]:
# Setup a temporary cache file. Only for
# demo purposes, use a known path in prod.
cache_data_path = pathlib.Path(tempfile.mkdtemp()) / "cache.pkl"

# Load a dataframe
df = goli.data.load_tiny_zinc()
df.head()

# Config for datamodule
dm_args = dict(config.data.args)
dm_args["df"] = df
dm_args["cache_data_path"] = cache_data_path
dm_args["featurization_n_jobs"] = 0
dm_args["featurization_progress"] = True

dm = goli.data.DGLFromSmilesDataModule(**dm_args)
dm

name: DGLFromSmilesDataModule
len: 100
train_val_batch_size: 16
test_batch_size: 16
num_node_feats: 50
num_edge_feats: 6
collate_fn: goli_collate_fn
featurization:
  atom_property_list_float: []
  atom_property_list_onehot:
  - atomic-number
  - degree
  edge_property_list:
  - ring
  - bond-type-onehot
  add_self_loop: false
  use_bonds_weights: false
  explicit_H: false

In [4]:
# Load and prepare the data
dm.prepare_data()

# Create the split torch datasets
dm.setup()

2021-03-17 09:56:22.856 | INFO     | goli.data.datamodule:prepare_data:174 - Prepare dataset with 100 data points.


  0%|          | 0/100 [00:00<?, ?it/s]

2021-03-17 09:56:23.399 | INFO     | goli.data.datamodule:prepare_data:206 - Write prepared data to /tmp/tmpon2zdfcq/cache.pkl


In [5]:
# Load a dataloader and get the first batch from it
dl = dm.train_dataloader()
it = iter(dl)
batch = next(it)
batch

{'smiles': ['COC(=O)c1cncc(C(=O)Nc2cccc(COC(C)C)c2)c1',
  'O=C(Nc1ccnn1Cc1cccc(Cl)c1)C1CCN(S(=O)(=O)c2ccccc2F)CC1',
  'Cc1ccc(C(=O)N2CCC[C@@H](C(=O)N(C)CC(=O)NC(C)C)C2)cc1',
  'COc1cc(F)cc(CNC(=O)[C@H]2CCCN2C(=O)Cc2ccccc2)c1',
  'CC(C)(C)OC(=O)N[C@H]1CCN(c2cc(-c3cccs3)n[nH]2)C1',
  'Cc1cc(CC(=O)N[C@H](c2ccc(F)cc2)C2CCC2)no1',
  'CCc1cc(C#N)c(NC(=O)CCC(=O)N(CC)CC)s1',
  'c1cc2c(cc1N[C@@H]1CCOC3(CCC3)C1)CCC2',
  'COCC[NH+](C)Cc1c(C)cc(C)c(C(C)=O)c1C',
  'Cc1ccc(-c2nc3ccc(C)c(C)c3[nH]2)nc1',
  'COc1cc(OC)c([C@@H](C)[NH2+]C[C@H](O)C2CCOCC2)cc1Cl',
  'Cc1ccsc1C(=O)NCCNC(=O)c1ccco1',
  'CC#CCCC(=O)Nc1cccc2c1C(=O)c1ccccc1C2=O',
  'Cc1n[nH]c(SCC(=O)Nc2cccc(C#Cc3ccccn3)c2)n1',
  'C[C@@H](C#N)Sc1nnc(C23CC4CC(CC(C4)C2)C3)o1',
  'CCc1ccc(C(=O)/C(=C(/S)NC2CC2)[n+]2ccc(CC)cc2)cc1'],
 'features': Graph(num_nodes=367, num_edges=788,
       ndata_schemes={'feat': Scheme(shape=(50,), dtype=torch.float32)}
       edata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}),
 'labels': tensor([[2.0442